In [1]:
%pylab inline

from datetime import datetime, timedelta
import re

from helpers import get_clarity_conn, get_mongo_conn, get_res_with_values, get_res_dict, chunk_res_with_values
from sqlalchemy.sql import text
from sqlalchemy.sql.expression import bindparam
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style('darkgrid')
sns.set_context('talk')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Populating the interactive namespace from numpy and matplotlib


In [2]:
clar_conn = get_clarity_conn()

In [5]:
q = 'select top 10 * from PAT_ENC;'
res = pd.DataFrame(get_res_dict(q, clar_conn))
res.head()

,ACCOUNT_ID,ACCREDITATION_C,ADM_FOR_SURG_YN,AGE,APPT_BLOCK_C,APPT_CANCEL_DATE,APPT_CANC_USER_ID,APPT_CONF_INST,APPT_CONF_PERS,APPT_CONF_STAT_C,APPT_ENTRY_USER_ID,APPT_LENGTH,APPT_MADE_DATE,APPT_PRC_ID,APPT_QNR_ANS_ID,APPT_SERIAL_NO,APPT_STATUS_C,APPT_TARGET_DATE,APPT_TIME,ARVL_LST_DL_TIME,ARVL_LST_DL_USR_ID,AR_EPISODE_ID,ASGND_INTERP_TYPE_C,ATTND_PROV_ID,AVS_FIRST_USER_ID,AVS_PRINT_TM,BENEFIT_COMP_ID,BENEFIT_PACKAGE_ID,BEN_ADJ_COPAY_AMT,BEN_ADJ_FORMULA_ID,BEN_ADJ_METHOD_C,BEN_ADJ_TABLE_ID,BEN_ENG_SP_AMT,BMI,BP_DIASTOLIC,BP_SYSTOLIC,BSA,CALCULATED_ENC_STAT_C,CANCEL_REASON_C,CANCEL_REASON_CMT,CERTIFICATION_C,CHARGE_SLIP_NUMBER,CHECKIN_TIME,CHECKIN_USER_ID,CHECKOUT_TIME,CHECK_IN_KIOSK_ID,CHKIN_INDICATOR_C,CHKIN_INDICATOR_DT,CLAIM_ID,CM_CT_OWNER_ID,CONSENT_TYPE_C,CONTACT_COMMENT,CONTACT_DATE,COPAY_COLLECTED,COPAY_DUE,COPAY_PD_THRU,COPAY_PMT_EXPL_C,COPAY_REF_NUM,COPAY_SOURCE_C,COPAY_TYPE_C,COVERAGE_ID,DATA_ENTRY_PERSON,DEPARTMENT_ID,DISCHARGE_DATE_DT,DOWNTIME_CSN,ED_ARRIVAL_KIOSK_ID,EFFECTIVE_DATE_DT,EFFECTIVE_DATE_DTTM,EFFECTIVE_DEPT_ID,ENC_CLOSED_USER_ID,ENC_CLOSED_YN,ENC_CLOSE_DATE,ENC_CLOSE_TIME,ENC_CREATE_USER_ID,ENC_INSTANT,ENC_MED_FRZ_RSN_C,ENC_TYPE_C,ENC_TYPE_TITLE,ENTITY_C,ENTRY_TIME,ES_ORDER_STATUS_C,EXTERNAL_VISIT_ID,FIN_CLASS_C,HEAD_CIRCUMFERENCE,HEIGHT,HOSP_ADMSN_TIME,HOSP_ADMSN_TYPE_C,HOSP_DISCHRG_TIME,HOSP_LICENSE_C,HSP_ACCOUNT_ID,INPATIENT_DATA_ID,INTERPRETER_NAME,INTERPRETER_NEED_YN,INTERPRETER_VEND_C,INTRP_ASSIGNMENT_C,IP_EPISODE_ID,IS_WALK_IN_YN,LMP_DATE,LMP_OTHER_C,LOS_MODIFIER1_ID,LOS_MODIFIER2_ID,LOS_MODIFIER3_ID,LOS_MODIFIER4_ID,LOS_PRIME_PROC_ID,LOS_PROC_CODE,NONCVRED_SERVICE_YN,ORDERING_PROV_ID,ORDERING_PROV_TEXT,OUTGOING_CALL_YN,PAT_ENC_CSN_ID,PAT_ENC_DATE_REAL,PAT_ID,PCP_PROV_ID,PHONE_REM_STAT_C,PHON_CALL_YN,PHON_NUM_APPT,PRIMARY_LOC_ID,PULSE,REFERRAL_ID,REFERRAL_REQ_YN,REFERRAL_SOURCE_ID,RESPIRATIONS,ROUTE_SUM_PRNT_YN,SERV_AREA_ID,SIGN_IN_TIME,SIGN_IN_USER_ID,SURGICAL_SVC_C,TEMPERATURE,TOBACCO_USE_VRFY_YN,UPDATE_DATE,VISIT_EPM_ID,VISIT_EPP_ID,VISIT_FC,VISIT_PROV_ID,VISIT_PROV_TITLE,VST_SPECIAL_NEEDS_C,WC_TPL_VISIT_C,WC_TPL_VISIT_CMT,WEIGHT,X_AVS_PRINT_TM,X_ENC_CLOSE_TIME
0,None,None,None,NaN,None,None,None,None,None,None,None,NaN,None,None,None,None,NaN,None,NaT,None,None,None,None,None,None,NaT,None,None,None,None,None,None,None,None,None,None,None,2,None,None,None,None,NaT,None,None,None,None,None,None,None,None,None,2010-11-23,None,None,None,None,None,None,None,None,None,76,None,None,None,2010-11-23,2010-11-23,76,HENDERSC,Y,2010-11-29,2010-11-29 13:58:00,KNIGHTB,2010-11-23 18:36:00,10.0,32002,None,None,2010-11-23 18:36:00,None,None,None,None,None,None,None,None,None,None,7334799,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,28633558,62053.02,059134296,RF8641,None,None,None,None,None,None,None,None,None,None,1000,None,None,None,None,None,2019-06-27 11:11:00,None,None,None,R02735,None,None,None,None,None,NaT,2010-11-29 13:58:00
1,None,None,None,0.44,None,None,None,None,None,None,None,NaN,None,None,None,None,NaN,None,NaT,None,None,None,None,None,None,NaT,None,None,None,None,None,None,None,None,None,None,None,2,None,None,None,None,NaT,None,None,None,None,None,None,None,None,None,2010-11-23,None,None,None,None,None,None,None,None,None,11,None,None,None,2010-11-23,2010-11-23,11,HARRISON,Y,2010-11-23,2010-11-23 18:39:00,HARRISON,2010-11-23 18:36:00,9.0,70,Telephone,None,2010-11-23 18:36:00,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,28633559,62053.00,Z42227,None,None,None,None,None,None,None,None,None,None,None,1000,None,None,None,None,None,2012-06-10 07:28:00,None,None,4,R00833,None,None,None,None,None,NaT,2010-11-23 18:39:00
2,None,None,None,70.00,None,None,None,None,None,None,None,NaN,None,None,None,None,NaN,None,NaT,None,None,None,None,None,None,NaT,None,None,None,None,None,None,None,None,None,None,None,2,None,None,None,None,NaT,None,None,None,None,None,None,None,

In [ ]:
q = text("""SELECT e.PAT_ENC_CSN_ID,
  mrn.IDENTITY_ID,
  icd10.CODE,
  CE.DX_NAME,
  e.ENTRY_TIME,
  e.EFFECTIVE_DATE_DTTM,
  'PAT_ENC_DX' as SOURCE
FROM PAT_ENC_DX PED
  INNER JOIN PAT_ENC e ON e.PAT_ENC_CSN_ID = PED.PAT_ENC_CSN_ID
  INNER JOIN CLARITY_EDG CE ON PED.DX_ID = CE.DX_ID
  inner join EDG_CURRENT_ICD10 icd10 on icd10.DX_ID = CE.DX_ID
  left join IDENTITY_ID mrn ON mrn.PAT_ID = e.PAT_ID AND mrn.IDENTITY_TYPE_ID = 100
WHERE icd10.CODE is not null and e.ENTRY_TIME > '2017-01-01' AND 
  (icd10.CODE like '%J44%' OR icd10.CODE like '%J43%' OR icd10.CODE like '%J41%' OR icd10.CODE like '%J42%' OR icd10.CODE like '%J84%' OR icd10.CODE like '%D86%' OR icd10.CODE like '%J84%' OR icd10.CODE like '%M34%' OR icd10.CODE like '%M23%')
  
UNION

select
  e.PAT_ENC_CSN_ID,
  mrn.IDENTITY_ID,
  icd10.CODE,
  CE.DX_NAME,
  e.ENTRY_TIME,
  e.EFFECTIVE_DATE_DTTM,
  'HSP_ACCT_DX_LIST' as SOURCE
FROM HSP_ACCT_DX_LIST PED
  LEFT JOIN CLARITY_EDG CE ON PED.DX_ID = CE.DX_ID
  LEFT JOIN ZC_DX_CC_HA z1 on z1.DX_CC_HA_C = ped.DX_COMORBIDITY_C
  LEFT JOIN ZC_DX_POA z2 on z2.DX_POA_C = ped.FINAL_DX_POA_C
  inner join EDG_CURRENT_ICD10 icd10 on icd10.DX_ID = CE.DX_ID
  LEFT JOIN PAT_ENC e on e.HSP_ACCOUNT_ID = ped.HSP_ACCOUNT_ID
  left join IDENTITY_ID mrn ON mrn.PAT_ID = e.PAT_ID AND mrn.IDENTITY_TYPE_ID = 100
WHERE
  icd10.CODE is not null and e.ENTRY_TIME > '2017-01-01' AND 
  (icd10.CODE like '%J44%' OR icd10.CODE like '%J43%' OR icd10.CODE like '%J41%' OR icd10.CODE like '%J42%' OR icd10.CODE like '%J84%' OR icd10.CODE like '%D86%' OR icd10.CODE like '%J84%' OR icd10.CODE like '%M34%' OR icd10.CODE like '%M23%')

UNION

select
  e.PAT_ENC_CSN_ID,
  mrn.IDENTITY_ID,
  icd10.CODE,
  edg.DX_NAME,
  e.ENTRY_TIME,
  e.EFFECTIVE_DATE_DTTM,
  'PROBLEM_LIST' as SOURCE
from
  PAT_ENC e
INNER JOIN PROBLEM_LIST_HX ph on ph.HX_PROBLEM_EPT_CSN = e.PAT_ENC_CSN_ID
inner join PROBLEM_LIST l on l.PROBLEM_LIST_ID = ph.PROBLEM_LIST_ID
inner join CLARITY_EDG edg on edg.DX_ID = ph.HX_PROBLEM_ID
inner join EDG_CURRENT_ICD10 icd10 on icd10.DX_ID = edg.DX_ID
left join ZC_DX_POA z1 on z1.DX_POA_C = ph.HX_PROBLEM_POA_C
left join IDENTITY_ID mrn ON mrn.PAT_ID = e.PAT_ID AND mrn.IDENTITY_TYPE_ID = 100

WHERE
  icd10.CODE is not null and e.ENTRY_TIME > '2017-01-01' AND 
  (icd10.CODE like '%J44%' OR icd10.CODE like '%J43%' OR icd10.CODE like '%J41%' OR icd10.CODE like '%J42%' OR icd10.CODE like '%J84%' OR icd10.CODE like '%D86%' OR icd10.CODE like '%J84%' OR icd10.CODE like '%M34%' OR icd10.CODE like '%M23%')""")
all_dx = pd.DataFrame(get_res_dict(q, clar_conn))
all_dx.shape

In [ ]:
DATA_DIR = '/example'

In [ ]:
all_dx = pd.read_json('{}/all_dx_2018-01-01_2019-10-25.json.bz2'.format(DATA_DIR))

In [ ]:
all_dx.shape

In [ ]:
all_dx.head()

In [ ]:
all_dx.to_json('{}/all_dx_2018-01-01_2019-10-25.json'.format(DATA_DIR))

In [ ]:
all_dx.columns = ['CODE', 'DX_NAME', 'EFFECTIVE_DATE_DTTM', 'ENTRY_TIME', 'MRN', 'PAT_ENC_CSN_ID', 'SOURCE']

In [ ]:
all_dx.head()

In [ ]:
chronic_re = '^(J44\.[0,1,9]|J43\.[0,1,2,8,9]|J41\.[0,1,8]|J42|J84\.10|D86|J84|M34\.81|M23)'

In [ ]:
chronic_dx = all_dx[all_dx['CODE'].str.contains(chronic_re)]#['PAT_ENC_CSN_ID'].unique()
chronic_dx.head()

In [ ]:
chronic_dx.shape

In [ ]:
chronic_dx['DX_NAME'].value_counts()

In [ ]:
chronic_dx['CODE'].value_counts()

In [ ]:
print(list([int(a) for a in chronic_dx.sample(10)['PAT_ENC_CSN_ID']]))

In [ ]:
print(list([a for a in chronic_dx.sample(20)['MRN']]))

In [ ]:
l = [int(a) for a in chronic_dx['PAT_ENC_CSN_ID'].unique()]
q = text("""SELECT e.PAT_ENC_CSN_ID
     , mrn.IDENTITY_ID MRN
     , e.EFFECTIVE_DATE_DTTM
     , e.ENTRY_TIME
     , datediff(day, pat.BIRTH_DATE, e.ENTRY_TIME) / 365.0      AGE
     , datediff(hour, hsp.HOSP_ADMSN_TIME, hsp.HOSP_DISCH_TIME) / 24.0 LOS
     , ZHAT.NAME           AS                                          ADMSN_TYPE
     , ser.NAME            as                                          SERVICE_NAME
     , zpc.NAME            as                                          PATIENT_CLASS
     , ZDD.NAME            AS                                          DISCHARGE_DISPOSITION
     , dep.DEPT_ABBREVIATION UNIT
     , dep.DEPARTMENT_NAME
     , dep.EXTERNAL_NAME
     , dep.SPECIALTY
     , CL.LOCATION_ABBR
     ,zs.NAME SEX
     ,zm.NAME MARITAL_STATUS
     ,zr.NAME RELIGION
     ,zes.NAME EMPY_STAT
     ,zpr.NAME RACE
     ,ze.NAME ETHNICITY
     ,zl.NAME LANGUAGE
     ,zc.NAME COUNTY
     ,pat.ZIP
     ,ZDET.NAME ENCOUNTER_TYPE
FROM PAT_ENC e
  left join PAT_ENC_2 e2 on e2.PAT_ENC_CSN_ID = e.PAT_ENC_CSN_ID
       left join PAT_ENC_HSP hsp ON hsp.PAT_ENC_CSN_ID = e.PAT_ENC_CSN_ID
       INNER JOIN PATIENT pat ON pat.PAT_ID = e.PAT_ID
       LEFT JOIN ZC_PAT_CLASS zpc ON e2.ADT_PAT_CLASS_C = zpc.ADT_PAT_CLASS_C
       LEFT JOIN ZC_HOSP_ADMSN_TYPE ZHAT ON e.HOSP_ADMSN_TYPE_C = ZHAT.HOSP_ADMSN_TYPE_C
       LEFT JOIN ZC_PAT_STATUS zps ON hsp.ADT_PATIENT_STAT_C = zps.ADT_PATIENT_STAT_C
       LEFT JOIN ZC_DISCH_DISP ZDD ON hsp.DISCH_DISP_C = ZDD.DISCH_DISP_C
       LEFT JOIN CLARITY_DEP dep ON dep.DEPARTMENT_ID = e.DEPARTMENT_ID
       LEFT JOIN CLARITY_LOC CL ON dep.REV_LOC_ID = CL.LOC_ID
       LEFT JOIN ZC_PAT_SERVICE ser ON ser.HOSP_SERV_C = hsp.HOSP_SERV_C
       LEFT JOIN ZC_SEX zs on pat.SEX_C = zs.RCPT_MEM_SEX_C
       LEFT JOIN ZC_ETHNIC_GROUP ze on pat.ETHNIC_GROUP_C = ze.ETHNIC_GROUP_C
       LEFT JOIN ZC_MARITAL_STATUS zm on pat.MARITAL_STATUS_C = zm.MARITAL_STATUS_C
       LEFT JOIN ZC_RELIGION zr on pat.RELIGION_C = zr.RELIGION_C
       LEFT JOIN ZC_LANGUAGE zl on pat.LANGUAGE_C = zl.LANGUAGE_C
       LEFT JOIN ZC_FIN_CLASS zf on pat.DEF_FIN_CLASS_C = zf.FIN_CLASS_C
       LEFT JOIN ZC_VETERAN_STAT zv on pat.VETERAN_STATUS_C = zv.VETERAN_STATUS_C
       lEFT JOIN ZC_EMPY_STAT zes on pat.EMPY_STATUS_C = zes.EMPY_STAT_C
       LEFT JOIN ZC_COUNTY zc on pat.COUNTY_C = zc.COUNTY_C
       LEFT JOIN PATIENT_RACE pr on pr.PAT_ID = e.PAT_ID and pr.LINE = 1
       LEFT JOIN ZC_PATIENT_RACE zpr on pr.PATIENT_RACE_C = zpr.PATIENT_RACE_C
       left join IDENTITY_ID mrn ON mrn.PAT_ID = e.PAT_ID AND mrn.IDENTITY_TYPE_ID = 100
       left join ZC_DISP_ENC_TYPE ZDET ON e.ENC_TYPE_C = ZDET.DISP_ENC_TYPE_C
WHERE e.PAT_ENC_CSN_ID in :ids
AND e.ENTRY_TIME >= '2017-01-01'""")
pat_info = chunk_res_with_values(q, l, clar_conn)
pat_info.shape

In [ ]:
chronic_dx['PAT_ENC_CSN_ID'].nunique()

In [ ]:
chronic_dx['MRN'].nunique()

In [ ]:
pat_info.to_json('{}/pat_info_2018-01-01_2019-10-25.json.bz2'.format(DATA_DIR))

In [ ]:
counts = chronic_dx['MRN'].value_counts()
mrns = (counts[counts > 1]).index
mrns
chronic_dx[chronic_dx['MRN'].isin(mrns)]['MRN'].nunique()

In [ ]:
pat_info['ENCOUNTER_TYPE'].value_counts()

In [ ]:
pat_info[pat_info['MRN'] == '444788137']

In [ ]:
pat_info['MRN'].unique()

In [ ]:
chronic_dx['MRN']

In [ ]:
%%bash
bash share_ipynb_to_datalab.sh "andrew data pull.ipynb"